In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
km = pd.read_csv('./input/KPMG Tax Case - Data Set.csv')
km.head(3)

,Date,Title,Numac,Link NL
0,1/14/2020,"REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel établissant les modèles de formulaire visé à l'article 8, § 1er, de l'arrêté du Gouvernement de la Région de Bruxelles-Capitale du 19 janvier 2017 établissant les modalités du régime préférentiel applicable aux droits de succession en cas de transmission des entreprises familiales et des sociétés familiales, p. 997.",2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.pl?language=nl&pub_date=2020-01-14&caller=list&numac=2020010053
1,1/16/2020,"SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE 2017. - Arrêté royal modifiant l'arrêté royal n° 20 du 20 juillet 1970, fixant les taux de la taxe sur la valeur ajoutée et déterminant la répartition des biens et des services selon ces taux en ce qui concerne certains produits destinés à la protection hygiénique intime et les défibrillateurs externes. - Traduction allemande, p. 1296.",2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.pl?language=nl&pub_date=2020-01-16&caller=list&numac=2020010044
2,1/16/2020,"SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 2018. - Arrêté royal modifiant l'AR/CIR 92, en ce qui concerne les avantages de toute nature résultant de la disposition gratuite d'immeubles ou de parties d'immeubles. - Traduction allemande, p. 1297.",2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article.pl?language=nl&pub_date=2020-01-16&caller=list&numac=2020040052


In [3]:
# Drop all columns except for the title
km.drop(['Date','Link NL'], axis=1,  inplace=True)

In [4]:
km1 = km.sort_values(by ='Numac' )

In [5]:
km1.drop([0,923], axis=0, inplace=True)

In [6]:
km2 = km1.copy()
# km2.head(3)

In [7]:
def preprocess(document_path):
    with open(document_path, 'r') as file:
        document = file.read().split('\n')
    return " ".join(document).lower()

In [8]:

Data = []
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        doc_num = filename
        cont = preprocess(os.path.join(dirname, filename))

        cont = re.sub('_______',' ',cont)
        cont = cont.replace(':','').replace('-','').replace('  ',' ').replace('.','')
        cont = cont.replace('§','').replace('آ°','').replace('آ','').replace('أ©','').replace('»',
                '').replace('أ¯','').replace('أ¶','ö').replace('‰','é').replace('أ©أ©','éé')
        cont = cont.replace(',','').replace(';',' ').replace('c)','').replace('b)','').replace('(','')
        cont = cont.replace("ذگ","").replace("رڈ","ï").replace("أ³أ³","óó").replace("أں",
               "ß").replace("أ¼","ü").replace("أœ","Ü").replace('«','').replace(')','').replace('(','')
        cont = re.sub(r'[0-9]',"", cont)
        cont = re.sub(r'[/]',"", cont)
        cont = re.sub('[a-z]*@','ABC@', cont)

        num = doc_num.replace(".txt","")
        
        Data.append([num ,cont])

df = pd.DataFrame (Data, columns = ['Num','Content'])
# df.head(3)

In [9]:
summar = []
for dirnam, _, filename in os.walk('./summaries_txt/'):
    for filenam in filename:
        doc_num1 = filenam
        cont1 = preprocess(os.path.join(dirnam, filenam))

        cont1 = cont.replace(':','').replace('-','')
        cont1 = re.sub(r'[0-9]',"", cont1)
        cont1 = re.sub(r'[/]',"", cont1)
        cont1 = re.sub('[a-z]*@','ABC@', cont1)

        numm = doc_num1.replace(".txt","")
        
        summar.append([numm ,cont1])

df_s = pd.DataFrame (summar, columns = ['NumTxt','Summary'])
# df_s.head(3)

In [10]:
print('df',df.info())
print('**'*20)
print('km',km.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num      768 non-null    object
 1   Content  768 non-null    object
dtypes: object(2)
memory usage: 12.1+ KB
df None
****************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926 entries, 0 to 925
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   926 non-null    object 
 1   Numac   910 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.6+ KB
km None


In [11]:
df2 = df.join(df_s)
# df1.sample(3)

In [12]:
df2.drop(['NumTxt'], axis=1,  inplace=True)

In [13]:
df2.count()

Num        768
Content    768
Summary    768
dtype: int64

In [14]:
df2.sample(2)

Num                                            Content  \
432  2021021049  publicatie numac  federale overheidsdienst bel...   
666  2021042174  publicatie numac  federale overheidsdienst fin...   

                                               Summary  
432  publicatie numac  federale overheidsdienst wer...  
666  publicatie numac  federale overheidsdienst wer...

### Pre-processing

In [15]:
import pprint

# df2 = pd.read_csv('./input/Data_with_summary.csv')
# df2.head(2)

In [18]:
sample = 767
pprint.pprint("Summary:{}".format(km['Title'][sample]))
pprint.pprint("Content:{}".format(df2['Summary'][sample][:767]))


('Summary:REGION DE BRUXELLES-CAPITALE\n'
 'REGION DE BRUXELLES-CAPITALE\n'
 '15 JUILLET 2021. - Arrêté du Gouvernement de la Région de Bruxelles-Capitale '
 'relatif à une aide aux organisations culturelles et créatives à caractère '
 'non lucratif dans le cadre de la crise sanitaire du COVID-19, p. 82965.')
('Content:publicatie numac  federale overheidsdienst werkgelegenheid arbeid en '
 'sociaal overleg   maart  koninklijk besluit tot verlenging van diverse '
 'maatregelen genomen op vlak van tijdelijke werkloosheid in het kader van de '
 'strijd tegen de verspreiding van het coronavirus covid naar aanleiding van '
 'de oorlog in oekrane en tot wijziging van artikel  van de wet van  december  '
 'houdende tijdelijke ondersteuningsmaatregelen ten gevolge van de '
 'covidpandemie filip koning der belgen aan allen die nu zijn en hierna wezen '
 'zullen onze groetgelet op de grondwet artikel  gelet op de besluitwet van  '
 'december  betreffende de maatschappelijke zekerheid der arbeide

In [19]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('dutch'))
## a list of Dutch stopwords
new_words = ["aan","af","al","als","bij","dan","dat","die","dit","een","en","er","had","heb","hem","het","hij","hoe","hun",
             "ik","in","is","je","kan","me","men","met","mij","nog","nu","of","ons","ook","te","tot","uit","van","was","wat",
             "we","wel","wij","zal","ze","zei","zij","zo","zou","aangaande","aangezien","achter","achterna","afgelopen",
             "aldaar","aldus","alhoewel","alias","alle","allebei","alleen","alsnog","altijd","altoos","ander","andere",
             "anders","anderszins","behalve","behoudens","beide","beiden","ben","beneden","bent","bepaald","betreffende",
             "binnen","binnenin","boven","bovenal","bovendien","bovengenoemd","bovenstaand","bovenvermeld","buiten","daar",
             "daarheen","daarin","daarna","daarnet","daarom","daarop","daarvanlangs","de","dikwijls","door","doorgaand","dus",
             "echter","eer","eerdat","eerder","eerlang","eerst","elk","elke","enig","enigszins","enkel","erdoor","even","eveneens",
             "evenwel","gauw","gedurende","geen","gehad","gekund","geleden","gelijk","gemoeten","gemogen","geweest","gewoon","gewoonweg",
             "haar","hadden","hare","hebben","hebt","heeft","hen","hierbeneden","hierboven","hoewel","hunne","ikzelf","inmiddels","inzake",
             "jezelf","jij","jijzelf","jou","jouw","jouwe","juist","jullie","klaar","kon","konden","krachtens","kunnen","kunt","later","liever",
             "maar","mag","meer","mezelf","mijn","mijnent","mijner","mijzelf","misschien","mocht","mochten","moest","moesten","moet","moeten","mogen",
             "na","naar","nadat","net","niet","noch","nogal","ofschoon","om","omdat","omhoog","omlaag","omstreeks","omtrent","omver","onder",
             "ondertussen","ongeveer","onszelf","onze","op","opnieuw","opzij","over","overeind","overigens","pas","precies","reeds","rond",
             "rondom","sedert","sinds","sindsdien","slechts","sommige","spoedig","steeds","tamelijk","tenzij","terwijl","thans","tijdens",
             "toch","toen","toenmaals","toenmalig","totdat","tussen","uitgezonderd","vaakwat","vandaan","vanuit","vanwege","veeleer","verder",
             "vervolgens","vol","volgens","voor","vooraf","vooral","vooralsnog","voorbij","voordat","voordezen","voordien","voorheen","voorop",
             "vooruit","vrij","vroeg","waar","waarom","wanneer","want","waren","weer","weg","wegens","weldra","welk","welke","wie","wiens","wier",
             "wijzelf","zelfs","zichzelf","zijn","zijne","zodra","zonder","zouden","zowat","zulke","zullen","zult"]

stop_words = list(stop_words.union(new_words))

def process(text):
    
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 2]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
    return text

pprint.pprint("Title:{}".format(km['Title'][sample]))
pprint.pprint("Summary:{}".format(df2['Summary'][sample][:767]))



km['Title'] = km['Title']
df['Summary'] = df2['Summary'].apply(lambda x:process(x))


df['Summary'][2]

('Title:REGION DE BRUXELLES-CAPITALE\n'
 'REGION DE BRUXELLES-CAPITALE\n'
 '15 JUILLET 2021. - Arrêté du Gouvernement de la Région de Bruxelles-Capitale '
 'relatif à une aide aux organisations culturelles et créatives à caractère '
 'non lucratif dans le cadre de la crise sanitaire du COVID-19, p. 82965.')
('Summary:publicatie numac  federale overheidsdienst werkgelegenheid arbeid en '
 'sociaal overleg   maart  koninklijk besluit tot verlenging van diverse '
 'maatregelen genomen op vlak van tijdelijke werkloosheid in het kader van de '
 'strijd tegen de verspreiding van het coronavirus covid naar aanleiding van '
 'de oorlog in oekrane en tot wijziging van artikel  van de wet van  december  '
 'houdende tijdelijke ondersteuningsmaatregelen ten gevolge van de '
 'covidpandemie filip koning der belgen aan allen die nu zijn en hierna wezen '
 'zullen onze groetgelet op de grondwet artikel  gelet op de besluitwet van  '
 'december  betreffende de maatschappelijke zekerheid der arbeiders

[]

In [20]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

#create a vocabulary of words 
cv = CountVectorizer(max_df=0.95,         
                   max_features=10000,  
                   ngram_range=(1,3)    
                  )
word_count_vector=cv.fit_transform(df2)

CPU times: total: 0 ns
Wall time: 2 ms


In [22]:
df2.columns

Index(['Num', 'Content', 'Summary'], dtype='object')